In [1]:
import pandas as pd
data_train = pd.read_csv('sarcasm_mal_train.csv')
data_val = pd.read_csv('sarcasm_mal_dev.csv')
data_test = pd.read_csv('sarcasm_mal_test_without_labels.csv')

In [2]:
text_train = data_train['Text']
text_val = data_val['Text']

class_label_train = data_train['labels']
class_label_val = data_val['labels']

text_test = data_test['Text']

In [3]:
class_label_train.value_counts(), class_label_val.value_counts()

(labels
 Non-sarcastic    10689
 Sarcastic         2499
 Name: count, dtype: int64,
 labels
 Non-sarcastic    2305
 Sarcastic         521
 Name: count, dtype: int64)

In [4]:
import numpy as np
classes_list = ["Non-sarcastic", "Sarcastic"]
label_index_train = class_label_train.apply(classes_list.index)
label_index_val = class_label_val.apply(classes_list.index)

In [5]:
import numpy as np
import ktrain
from ktrain import text

In [6]:
MODEL_NAME = 'roberta-base'
t = text.Transformer(MODEL_NAME, maxlen=30, classes=classes_list)
trn = t.preprocess_train(np.array(text_train), np.array(class_label_train))
val = t.preprocess_test(np.array(text_val), np.array(class_label_val))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

c:\Users\nimmi\anaconda3\Lib\site-packages\ktrain\text\preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

c:\Users\nimmi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nimmi\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

preprocessing train...
language: ml
train sequence lengths:
	mean : 10
	95percentile : 21
	99percentile : 47


...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: ml
test sequence lengths:
	mean : 10
	95percentile : 21
	99percentile : 45


In [7]:
learner.fit_onecycle(5e-5, 15) 



begin training using onecycle policy with max lr of 5e-05...


Epoch 1/15
413/413 [==============================] - 3317s 8s/step - loss: 0.4665 - accuracy: 0.8107 - val_loss: 0.4410 - val_accuracy: 0.8220
Epoch 2/15
413/413 [==============================] - 3123s 8s/step - loss: 0.4364 - accuracy: 0.8178 - val_loss: 0.4192 - val_accuracy: 0.8234
Epoch 3/15
413/413 [==============================] - 3134s 8s/step - loss: 0.4230 - accuracy: 0.8194 - val_loss: 0.4173 - val_accuracy: 0.8227
Epoch 4/15
413/413 [==============================] - 3147s 8s/step - loss: 0.3987 - accuracy: 0.8284 - val_loss: 0.4026 - val_accuracy: 0.8287
Epoch 5/15
413/413 [==============================] - 3141s 8s/step - loss: 0.3842 - accuracy: 0.8330 - val_loss: 0.3995 - val_accuracy: 0.8351
Epoch 6/15
413/413 [==============================] - 3032s 7s/step - loss: 0.3597 - accuracy: 0.8401 - val_loss: 0.4318 - val_accuracy: 0.8163
Epoch 7/15
413/413 [==============================] - 14591s 35s/step - loss: 0.3441 - accuracy: 0.8423 - val_loss: 0.4363 - val_accurac

In [8]:
learner.validate(class_names=t.get_classes()) 

89/89 [==============================] - 245s 2s/step
               precision    recall  f1-score   support

Non-sarcastic       0.86      0.94      0.90      2305
    Sarcastic       0.56      0.32      0.41       521

     accuracy                           0.83      2826
    macro avg       0.71      0.63      0.66      2826
 weighted avg       0.81      0.83      0.81      2826



array([[2174,  131],
       [ 352,  169]], dtype=int64)

In [9]:
learner.view_top_losses(n=1, preproc=t)

89/89 [==============================] - 202s 2s/step
----------
id:1394 | loss:7.73 | true:Sarcastic | pred:Non-sarcastic)



In [10]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [11]:
predictor.predict(text_test[0])

'Non-sarcastic'

In [12]:
predictor.predict_proba(text_test[0])

array([9.9955434e-01, 4.4566597e-04], dtype=float32)

In [13]:
predictor.get_classes()

['Non-sarcastic', 'Sarcastic']

In [14]:
predictor.explain(text_test[0])

In [15]:
text_test[1]

'ഗീതു മോഹൻദാസ് മലയാള സിനിമക്കു നൽകുന്ന വമ്പൻ ഗിഫ്റ്റ് തന്നെ ആവും മൂത്തോന്'

In [16]:
predictor.save('Roberta_Malayalam_SARCASM_DETECTION')

In [17]:
reloaded_predictor = ktrain.load_predictor('Roberta_Malayalam_SARCASM_DETECTION')

In [18]:
predict = reloaded_predictor.predict(text_test.values)

In [19]:
data_test['label'] = predict

In [20]:
new_data_test = data_test[['ID','label']]

In [21]:
import csv
new_data_test.to_csv("Malayalam_roberta_sarcasm_detection.tsv", sep="\t", quoting=csv.QUOTE_NONE)

In [22]:
check_submission = pd.read_csv("Malayalam_roberta_sarcasm_detection.tsv", sep="\t")

In [23]:
check_submission

,Unnamed: 0,ID,label
0,0,Id_01,Non-sarcastic
1,1,Id_02,Sarcastic
2,2,Id_03,Non-sarcastic
3,3,Id_04,Non-sarcastic
4,4,Id_05,Non-sarcastic
...,...,...,...
2821,2821,Id_2822,Non-sarcastic
2822,2822,Id_2823,Non-sarcastic
2823,2823,Id_2824,Non-sarcastic
2824,2824,Id_2825,Non-sarcastic


In [24]:
check_submission['label'].value_counts()

label
Non-sarcastic    2520
Sarcastic         306
Name: count, dtype: int64

In [25]:
model.summary()

Model: "tf_roberta_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124055040 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592130    
 ificationHead)                                                  
                                                                 
Total params: 124647170 (475.49 MB)
Trainable params: 124647170 (475.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
